<a href="https://colab.research.google.com/github/ShushanManandyan/Object-Detection-Pytorch/blob/main/YOLOX_Helmet_Training_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLOX Helmet Detection Training

This notebook trains YOLOX-L model on a construction safety helmet dataset.

**Dataset Classes (6):**
- construction-safety
- helmet
- no-helmet
- no-vest
- person
- vest

**Prerequisites:**
1. Upload your dataset to Google Drive
2. Enable GPU runtime: Runtime → Change runtime type → GPU (T4 or better)

In [1]:
# Step 1: Check GPU availability
!nvidia-smi

import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Sat Jan 17 12:17:35 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Step 2: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Step 3: Clone YOLOX repository
!git clone https://github.com/Megvii-BaseDetection/YOLOX.git
%cd YOLOX

Cloning into 'YOLOX'...
remote: Enumerating objects: 1940, done.
remote: Total 1940 (delta 0), reused 0 (delta 0), pack-reused 1940 (from 1)
Receiving objects: 100% (1940/1940), 7.55 MiB | 14.78 MiB/s, done.
Resolving deltas: 100% (1163/1163), done.
/content/YOLOX


In [4]:
# Step 4: Install YOLOX and dependencies
!pip install -v -e .
!pip install cython
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip install pycocotools

Using pip 24.1.2 from /usr/local/lib/python3.12/dist-packages/pip (python 3.12)
Obtaining file:///content/YOLOX
  Running command python setup.py egg_info
  /usr/local/lib/python3.12/dist-packages/setuptools/__init__.py:94: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
  !!

          ********************************************************************************
          Requirements should be satisfied by a PEP 517 installer.
          If you are using pip, you can try `pip install --use-pep517`.
          ********************************************************************************

  !!
    dist.fetch_build_eggs(dist.setup_requires)
  running egg_info
  creating /tmp/pip-pip-egg-info-ufnbkh5m/yolox.egg-info
  writing /tmp/pip-pip-egg-info-ufnbkh5m/yolox.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-ufnbkh5m/yolox.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-pip-egg-info-ufnbkh5m/yolox.egg-info/requir

In [5]:
# Step 5: Copy dataset from Google Drive
# IMPORTANT: Change the path to match your Google Drive folder
# Your dataset structure should be:
# helmet_dataset/
#   ├── train/
#   │   ├── _annotations.coco.json
#   │   └── *.jpg
#   ├── valid/
#   │   ├── _annotations.coco.json
#   │   └── *.jpg
#   └── test/
#       ├── _annotations.coco.json
#       └── *.jpg

!cp -r "/content/drive/MyDrive/helmet_dataset" /content/YOLOX/datasets/

# Verify dataset
!ls -la /content/YOLOX/datasets/helmet_dataset/

total 124
drwx------ 5 root root  4096 Jan 17 12:21 .
drwxr-xr-x 3 root root  4096 Jan 17 12:20 ..
-rw------- 1 root root   678 Jan 17 12:20 README.dataset.txt
-rw------- 1 root root   910 Jan 17 12:20 README.roboflow.txt
drwx------ 2 root root 12288 Jan 17 12:22 test
drwx------ 2 root root 86016 Jan 17 12:21 train
drwx------ 2 root root 12288 Jan 17 12:21 valid


In [6]:
# Step 6: Verify dataset annotations
import json

with open('/content/YOLOX/datasets/helmet_dataset/train/_annotations.coco.json') as f:
    data = json.load(f)

print("Dataset Info:")
print(f"  Categories: {[c['name'] for c in data['categories']]}")
print(f"  Number of training images: {len(data['images'])}")
print(f"  Number of annotations: {len(data['annotations'])}")

Dataset Info:
  Categories: ['construction-safety', 'helmet', 'no-helmet', 'no-vest', 'person', 'vest']
  Number of training images: 997
  Number of annotations: 6386


In [8]:
# Step 7: Create custom dataset class
dataset_code = '''
"""
Helmet Dataset class for YOLOX training.
"""

import os
import cv2
import numpy as np
from pycocotools.coco import COCO
from yolox.data.datasets.datasets_wrapper import Dataset


class HelmetDataset(Dataset):
    """
    COCO format dataset for helmet detection.
    """

    def __init__(
        self,
        data_dir,
        json_file="_annotations.coco.json",
        name="train",
        img_size=(640, 640),
        preproc=None,
    ):
        super().__init__(img_size)
        self.data_dir = data_dir
        self.json_file = json_file
        self.name = name
        self.img_size = img_size
        self.preproc = preproc

        self.coco = COCO(os.path.join(data_dir, name, json_file))
        self.ids = list(self.coco.imgs.keys())

        self.class_ids = sorted(self.coco.getCatIds())
        cats = self.coco.loadCats(self.class_ids)
        self.classes = tuple([c["name"] for c in cats])

        self.cat_to_class = {v: i for i, v in enumerate(self.class_ids)}
        self.annotations = self._load_coco_annotations()

    def __len__(self):
        return len(self.ids)

    def _load_coco_annotations(self):
        return [self.load_anno_from_ids(_id) for _id in self.ids]

    def load_anno_from_ids(self, id_):
        im_ann = self.coco.loadImgs(id_)[0]
        width = im_ann["width"]
        height = im_ann["height"]
        file_name = im_ann["file_name"]

        anno_ids = self.coco.getAnnIds(imgIds=[id_], iscrowd=False)
        annotations = self.coco.loadAnns(anno_ids)

        objs = []
        for obj in annotations:
            x1 = obj["bbox"][0]
            y1 = obj["bbox"][1]
            x2 = x1 + obj["bbox"][2]
            y2 = y1 + obj["bbox"][3]

            if obj["area"] > 0 and x2 > x1 and y2 > y1:
                obj["clean_bbox"] = [x1, y1, x2, y2]
                obj["class_id"] = self.cat_to_class[obj["category_id"]]
                objs.append(obj)

        num_objs = len(objs)
        res = np.zeros((num_objs, 5))

        for ix, obj in enumerate(objs):
            res[ix, 0:4] = obj["clean_bbox"]
            res[ix, 4] = obj["class_id"]

        img_info = (height, width)
        resized_info = img_info

        return (res, img_info, resized_info, file_name)

    def load_anno(self, index):
        return self.annotations[index][0]

    def load_resized_img(self, index):
        img = self.load_image(index)
        r = min(self.img_size[0] / img.shape[0], self.img_size[1] / img.shape[1])
        resized_img = cv2.resize(
            img,
            (int(img.shape[1] * r), int(img.shape[0] * r)),
            interpolation=cv2.INTER_LINEAR,
        ).astype(np.uint8)
        return resized_img

    def load_image(self, index):
        file_name = self.annotations[index][3]
        img_file = os.path.join(self.data_dir, self.name, file_name)
        img = cv2.imread(img_file)
        assert img is not None, f"Image not found: {img_file}"
        return img

    def pull_item(self, index):
        res, img_info, resized_info, _ = self.annotations[index]
        img = self.load_resized_img(index)
        return img, res.copy(), img_info, index

    def __getitem__(self, index):
        img, target, img_info, img_id = self.pull_item(index)

        if self.preproc is not None:
            img, target = self.preproc(img, target, self.input_dim)

        return img, target, img_info, img_id
'''

with open('/content/YOLOX/yolox/data/datasets/helmet_dataset.py', 'w') as f:
    f.write(dataset_code)

print("Dataset class created successfully!")

Dataset class created successfully!


In [9]:
# Step 8: Create training experiment configuration
import os
os.makedirs('/content/YOLOX/exps/helmet', exist_ok=True)

exp_code = '''
"""
YOLOX-L experiment configuration for helmet detection.
6 classes: construction-safety, helmet, no-helmet, no-vest, person, vest
"""

import os
import sys

sys.path.insert(0, '/content/YOLOX')

from yolox.exp import Exp as MyExp


class Exp(MyExp):
    def __init__(self):
        super(Exp, self).__init__()

        # Model configuration (YOLOX-L)
        self.depth = 1.0
        self.width = 1.0
        self.num_classes = 6

        # Input size
        self.input_size = (640, 640)
        self.test_size = (640, 640)

        # Training settings
        self.max_epoch = 50
        self.warmup_epochs = 3
        self.basic_lr_per_img = 0.01 / 64.0
        self.weight_decay = 5e-4
        self.momentum = 0.9

        # Data augmentation
        self.mosaic_prob = 1.0
        self.mixup_prob = 1.0
        self.hsv_prob = 1.0
        self.flip_prob = 0.5
        self.degrees = 10.0
        self.translate = 0.1
        self.mosaic_scale = (0.5, 1.5)
        self.mixup_scale = (0.5, 1.5)
        self.shear = 2.0

        self.no_aug_epochs = 5

        # Evaluation
        self.eval_interval = 5
        self.nmsthre = 0.65
        self.test_conf = 0.01

        # Experiment name
        self.exp_name = "helmet_yolox_l"

        # Dataset paths
        self.data_dir = "/content/YOLOX/datasets/helmet_dataset"
        self.train_ann = "_annotations.coco.json"
        self.val_ann = "_annotations.coco.json"

    def get_data_loader(self, batch_size, is_distributed, no_aug=False, cache_img=False):
        from yolox.data import TrainTransform
        from yolox.data.datasets.helmet_dataset import HelmetDataset
        from yolox.data import (
            YoloBatchSampler,
            DataLoader,
            InfiniteSampler,
            MosaicDetection,
            worker_init_reset_seed,
        )

        dataset = HelmetDataset(
            data_dir=self.data_dir,
            json_file=self.train_ann,
            name="train",
            img_size=self.input_size,
            preproc=TrainTransform(
                max_labels=100,
                flip_prob=self.flip_prob,
                hsv_prob=self.hsv_prob
            ),
        )

        dataset = MosaicDetection(
            dataset,
            mosaic=not no_aug,
            img_size=self.input_size,
            preproc=TrainTransform(
                max_labels=200,
                flip_prob=self.flip_prob,
                hsv_prob=self.hsv_prob
            ),
            degrees=self.degrees,
            translate=self.translate,
            mosaic_scale=self.mosaic_scale,
            mixup_scale=self.mixup_scale,
            shear=self.shear,
            mosaic_prob=self.mosaic_prob,
            mixup_prob=self.mixup_prob,
        )

        self.dataset = dataset

        sampler = InfiniteSampler(len(self.dataset), seed=self.seed if self.seed else 0)

        batch_sampler = YoloBatchSampler(
            sampler=sampler,
            batch_size=batch_size,
            drop_last=False,
            mosaic=not no_aug,
        )

        dataloader_kwargs = {"num_workers": 4, "pin_memory": True}
        dataloader_kwargs["batch_sampler"] = batch_sampler
        dataloader_kwargs["worker_init_fn"] = worker_init_reset_seed

        train_loader = DataLoader(self.dataset, **dataloader_kwargs)

        return train_loader

    def get_eval_loader(self, batch_size, is_distributed, testdev=False, legacy=False):
        from yolox.data import ValTransform
        from yolox.data.datasets.helmet_dataset import HelmetDataset
        from torch.utils.data import DataLoader

        valdataset = HelmetDataset(
            data_dir=self.data_dir,
            json_file=self.val_ann,
            name="valid",
            img_size=self.test_size,
            preproc=ValTransform(legacy=legacy),
        )

        sampler = None

        dataloader_kwargs = {
            "num_workers": 4,
            "pin_memory": True,
            "sampler": sampler,
            "batch_size": batch_size,
        }
        val_loader = DataLoader(valdataset, **dataloader_kwargs)

        return val_loader

    def get_evaluator(self, batch_size, is_distributed, testdev=False, legacy=False):
        from yolox.evaluators import COCOEvaluator

        val_loader = self.get_eval_loader(batch_size, is_distributed, testdev, legacy)
        evaluator = COCOEvaluator(
            dataloader=val_loader,
            img_size=self.test_size,
            confthre=self.test_conf,
            nmsthre=self.nmsthre,
            num_classes=self.num_classes,
            testdev=testdev,
        )
        return evaluator
'''

with open('/content/YOLOX/exps/helmet/helmet_yolox_l.py', 'w') as f:
    f.write(exp_code)

print("Experiment config created successfully!")

Experiment config created successfully!


In [10]:
# Step 9: Download YOLOX-L pre-trained weights
!mkdir -p /content/YOLOX/weights
!wget -O /content/YOLOX/weights/yolox_l.pth https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_l.pth

# Verify download
!ls -lh /content/YOLOX/weights/

--2026-01-17 12:28:41--  https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_l.pth
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/386811486/4e1d8aa7-93cc-494a-8a10-706db8bbc57b?sp=r&sv=2018-11-09&sr=b&spr=https&se=2026-01-17T13%3A18%3A16Z&rscd=attachment%3B+filename%3Dyolox_l.pth&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2026-01-17T12%3A17%3A39Z&ske=2026-01-17T13%3A18%3A16Z&sks=b&skv=2018-11-09&sig=fcwC%2F5jXC4Q2BAbs82AwaEvHZKZs3W9LKBDpFs%2BdOfc%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2ODY1NjUyMSwibmJmIjoxNzY4NjUyOTIxLCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvb

In [ ]:
# Step 10: Start Training
# Adjust batch_size based on GPU memory:
# - T4 (16GB): batch_size=8
# - V100 (16GB): batch_size=16
# - A100 (40GB): batch_size=32

!python /content/YOLOX/tools/train.py \
    -f /content/YOLOX/exps/helmet/helmet_yolox_l.py \
    -c /content/YOLOX/weights/yolox_l.pth \
    -d 1 \
    -b 8 \
    --fp16 \
    -o

In [ ]:
# Step 11 (Optional): Monitor training with TensorBoard
%load_ext tensorboard
%tensorboard --logdir /content/YOLOX/YOLOX_outputs/helmet_yolox_l/tensorboard

In [ ]:
# Step 12: Evaluate trained model on validation set
!python /content/YOLOX/tools/eval.py \
    -f /content/YOLOX/exps/helmet/helmet_yolox_l.py \
    -c /content/YOLOX/YOLOX_outputs/helmet_yolox_l/best_ckpt.pth \
    -d 1 \
    -b 8

In [ ]:
# Step 13: Run inference on test images
!python /content/YOLOX/tools/demo.py image \
    -f /content/YOLOX/exps/helmet/helmet_yolox_l.py \
    -c /content/YOLOX/YOLOX_outputs/helmet_yolox_l/best_ckpt.pth \
    --path /content/YOLOX/datasets/helmet_dataset/test/ \
    --conf 0.3 \
    --nms 0.65 \
    --save_result \
    --device gpu

In [ ]:
# Step 14: Display inference results
import matplotlib.pyplot as plt
import cv2
import glob

result_images = glob.glob('/content/YOLOX/YOLOX_outputs/helmet_yolox_l/vis_res/**/*.jpg', recursive=True)
print(f"Found {len(result_images)} result images")

if result_images:
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()

    for i, img_path in enumerate(result_images[:6]):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        axes[i].imshow(img)
        axes[i].axis('off')
        axes[i].set_title(f'Result {i+1}')

    plt.tight_layout()
    plt.show()

In [ ]:
# Step 15: Save trained model to Google Drive
!mkdir -p "/content/drive/MyDrive/yolox_helmet_model"

# Copy best checkpoint
!cp /content/YOLOX/YOLOX_outputs/helmet_yolox_l/best_ckpt.pth \
    "/content/drive/MyDrive/yolox_helmet_model/"

# Copy latest checkpoint
!cp /content/YOLOX/YOLOX_outputs/helmet_yolox_l/latest_ckpt.pth \
    "/content/drive/MyDrive/yolox_helmet_model/"

# Copy experiment config
!cp /content/YOLOX/exps/helmet/helmet_yolox_l.py \
    "/content/drive/MyDrive/yolox_helmet_model/"

# Copy dataset class
!cp /content/YOLOX/yolox/data/datasets/helmet_dataset.py \
    "/content/drive/MyDrive/yolox_helmet_model/"

print("Model saved to Google Drive!")
!ls -lh "/content/drive/MyDrive/yolox_helmet_model/"

In [ ]:
# Step 16: Download model directly to your computer
from google.colab import files
files.download('/content/YOLOX/YOLOX_outputs/helmet_yolox_l/best_ckpt.pth')

In [ ]:
# Step 17 (Optional): Export to ONNX format for deployment
!python /content/YOLOX/tools/export_onnx.py \
    -f /content/YOLOX/exps/helmet/helmet_yolox_l.py \
    -c /content/YOLOX/YOLOX_outputs/helmet_yolox_l/best_ckpt.pth \
    --output-name helmet_yolox_l.onnx

# Copy ONNX model to Google Drive
!cp helmet_yolox_l.onnx "/content/drive/MyDrive/yolox_helmet_model/"
print("ONNX model exported!")

## Training Tips

1. **GPU Memory**: Reduce batch size if you get OOM errors
   - T4 (16GB): batch_size=8
   - V100 (16GB): batch_size=16
   - A100 (40GB): batch_size=32

2. **Training Time**:
   - 50 epochs with ~1000 images takes ~2-4 hours on T4

3. **Resume Training**: If training is interrupted, use:
   ```
   !python /content/YOLOX/tools/train.py \
       -f /content/YOLOX/exps/helmet/helmet_yolox_l.py \
       --resume \
       --ckpt /content/YOLOX/YOLOX_outputs/helmet_yolox_l/latest_ckpt.pth \
       -d 1 -b 8 --fp16 -o
   ```

4. **Best Practices**:
   - Use `--fp16` for faster training with mixed precision
   - Monitor loss in TensorBoard
   - Save checkpoints to Google Drive periodically